# E-Commerce Logs Analysis

## Project Objective

The objective of this project is to analyze the website logs from an e-commerce store to obtain useful insight into customer behaviour.

## Data Source

The dataset was obtained here: https://www.kaggle.com/datasets/kzmontage/e-commerce-website-logs and the data has an Open Database license: https://opendatacommons.org/licenses/odbl/1-0/

In [1]:
-- Specifying database
USE EcommerceLogsDb

Commands completed successfully.

Total execution time: 00:00:00.026

In [2]:
-- preview of purchase logs table
SELECT TOP 3*
FROM logs;

(3 rows affected)

Total execution time: 00:00:00.502

access_date,duration_seconds,Proto,IP,Src_IP_type,Src_Pt,Bytes,Accessed_From,country,membership,Languages,Sales,Returned,Returned_Amount
2016-11-01 09:58:00.0000000,2533,TCP,1.10.195.126,EXT_SERVER,8082,20100,Chrome,Albania,Normal,Albanian,261.96,No,0
2016-11-01 09:59:00.0000000,4034,TCP,1.1.217.211,OPENSTACK_NET,56978,20500,Mozilla Firefox,Albania,Normal,Albanian,731.94,No,0
2016-11-01 09:59:00.0000000,1525,TCP,1.115.198.107,EXT_SERVER,8082,90100,Mozilla Firefox,Albania,Normal,Albanian,14.62,No,0


In [10]:
-- Which country had the highest number of sales and highest value of sales (in $)?
-- showing country percentiles for both metrics using window functions
SELECT TOP 5 country, 
    COUNT(*) AS num_sales, 
    PERCENT_RANK() OVER(ORDER BY COUNT(*)) AS num_sales_percentile, 
    SUM(Sales) AS value_sales, 
    PERCENT_RANK() OVER(ORDER BY SUM(Sales)) AS value_sales_percentile
FROM logs
GROUP BY country
ORDER BY value_sales DESC;

(5 rows affected)

Total execution time: 00:00:00.187

country,num_sales,num_sales_percentile,value_sales,value_sales_percentile
Austria,2674,1,1296463.3414999906,1
Belgium,2604,0.97,1154321.3705999977,0.99
Argentina,2603,0.96,1140238.6927999987,0.98
Australia,2520,0.94,1139576.1649999968,0.97
United States,2231,0.73,1132496.3800000001,0.96


From the table above, the top 4 countries for num\_sales and value\_sales are Austria, Belgium, Argentina and Australia. The number of sales seems highly correlated with value of sales with some exceptions like the United States which ranks in the 73rd percentile for number of sales and in the 96th percentile for value of sales.

Next let's look at the average order value (AOV) per country and compare to the worldwide average. The AOV is a popular e-commerce metric that indicates the average amount customers are spending in one purchase. Both the tradditional AOV (using the mean) and the median AOV will be calculated adnc compared.

In [11]:
-- average order value ($) per country
SELECT TOP 5 
    country, AVG(Sales) AS avg_order_value,
    -- subquery in SELECT to determine worldwide avg order value ($)
    (SELECT AVG(Sales) FROM logs) AS world_avg_order_value,
    -- difference bewteen country avg and worldwide avg with another SELECT subquery
    (AVG(Sales) - (SELECT AVG(Sales) FROM logs)) AS difference
FROM logs
GROUP BY country
ORDER BY difference DESC;

(5 rows affected)

Total execution time: 00:00:00.141

country,avg_order_value,world_avg_order_value,difference
Turkey,557.528015873016,411.346448791977,146.18156708102913
Norway,523.0527268518518,411.346448791977,111.70627805986499
Qatar,519.2086891385769,411.346448791977,107.86224034659006
Nicaragua,516.7759722222223,411.346448791977,105.42952343023546
Slovenia,516.1550165453344,411.346448791977,104.80856775334757


In [20]:
-- median order value ($) per country
WITH CTE_AOV AS (
    SELECT  
        DISTINCT country,
        PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY Sales) OVER(PARTITION BY country) AS median_AOV,
        PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY Sales) OVER() AS world_median_AOV
    FROM logs)
    --ORDER BY median_AOV DESC)

SELECT DISTINCT TOP 5 logs.country, median_AOV, AVG(median_AOV) OVER () AS avg_world_aov
FROM logs
    INNER JOIN CTE_AOV
    ON logs.country = CTE_AOV.country;

(5 rows affected)

Total execution time: 00:00:04.658

country,median_AOV,avg_world_aov
Kiribati,46.92,57.028022315694535
Netherlands,71.68,57.028022315694535
New Zealand,58.65,57.028022315694535
Greece,30.3,57.028022315694535
Nicaragua,81.32,57.028022315694535


The average order values (mean) range from $235-$557 and are much higher than the median AOV's which range from $10-$122. This indicates that there are several large outliers for each dataset that pull the mean much higher than the median. As a result, we will use the median AOV because it is more robust to outliers and is a better representation for central tendency in this case.

We'll now look at returns of purchased items for each country.

In [21]:
-- CTE for value of sales returned
WITH returned_value AS (
    SELECT country,
        -- Sum of total dollars returned
        SUM(Returned_Amount) AS returned_amount,
        -- Percentage of dollars returned over dollars sold
        -- Sales column doesn't include amount for returned items
        SUM(Returned_Amount) / (SUM(Sales) + SUM(Returned_Amount)) AS pct_value_returned
    From logs
    GROUP BY country),

-- CTE for number of sales returned
returned_sales AS (
    SELECT t1.country, 
        -- counting all rows (number of transactions)
        COUNT(*) AS num_sales,
        -- counting number of order returns 
        COUNT(t2.Returned) AS sales_returned,
        -- calculating percentage of orders that were returned
        1.0 * COUNT(t2.Returned) / COUNT(*) AS pct_sales_returned
    FROM logs as t1
    LEFT JOIN (SELECT duration_seconds, ip, country, Returned
    FROM logs 
    WHERE Returned = 'Yes') AS t2
    ON t1.duration_seconds = t2.duration_seconds
    AND t1.ip = t2.ip
    GROUP BY t1.country)

-- final query with percent of orders returned and percent of sales refunded
SELECT TOP 5 CTE1.country, pct_sales_returned, pct_value_returned 
FROM returned_value AS CTE1
    INNER JOIN returned_sales AS CTE2
    ON CTE1.country = CTE2.country 
ORDER BY pct_sales_returned DESC, pct_value_returned DESC;



Warning: Null value is eliminated by an aggregate or other SET operation.

(5 rows affected)

Total execution time: 00:00:00.196

country,pct_sales_returned,pct_value_returned
Nicaragua,0.212962962962,0.21179128180131443
Oman,0.212962962962,0.15195254761361243
Montenegro,0.190277777777,0.2197341784693778
Puerto Rico,0.184139784946,0.15605512400495514
Poland,0.182870370370,0.14169463345436825


Nicaragua and Oman have the highest percentage of orders returned at 21.3%. This represents a possible opportunity to increase revenue by making the return policy more strict to attempt to lower the number of returned orders. The affect that a stricter return policy would have on customer experience should also be taken into account though.

We will next look at the number of returns for each language. This will help inform us of any language/communication issues thay may be causing a higher rate of purchase returns.

In [7]:
SELECT TOP 5 t1.Languages, 
        -- calculating percentage of orders that were returned
        ROUND((1.0 * COUNT(t2.Returned) / COUNT(*)), 2) AS pct_orders_returned
FROM logs as t1
        LEFT JOIN (SELECT duration_seconds, ip, country, Returned
        FROM logs 
        WHERE Returned = 'Yes') AS t2
        ON t1.duration_seconds = t2.duration_seconds
        AND t1.ip = t2.ip
GROUP BY t1.Languages
ORDER BY pct_orders_returned DESC;

Warning: Null value is eliminated by an aggregate or other SET operation.

(5 rows affected)

Total execution time: 00:00:00.144

Languages,pct_orders_returned
marathi,0.220000000000
romanian,0.180000000000
kinyarwanda,0.180000000000
polish,0.170000000000
Slovak,0.170000000000


Marathi, a language spoken in India, has the highest percentage for orders returned (4% above the second language). A statistical test should be performed to determine if this percentage is statistically significantly greater than the 2nd highest percentage. If it is, this indicates that higher return rates could be linked to translation issues with the language. This means customers could be misinformed about products, resulting in higher return rates. The impact of language on return rates should be further investigated for the website as a whole to idenitfy if that is a frequent issue.

The membership types associated with online purchases will now be investigated per country. It is important to note that each count of a membership type represents an online purchase but not a unique individual (no unique identifiers are present in dataset). It is likely that premium members make more frequent purchases than normal and non-members. This category therefore investigates purchases per membership type and not individuals per membership type.

In [22]:
-- first CTE for total website purchases per country
WITH totals AS (
    SELECT country, COUNT(*) AS total_purchases
    FROM logs
    GROUP BY country),

-- second CTE for website purchases per membership type per country
memberships AS (
    SELECT country, membership, COUNT(1) AS website_purchase
    FROM logs
    GROUP BY country, membership)

-- joining CTEs and calculating pct membership type for total purchases per country
SELECT TOP 5 memberships.country, membership, website_purchase, 1.0 * website_purchase/total_purchases AS pct_visits
FROM memberships
    LEFT JOIN totals
    ON memberships.country = totals.country
ORDER BY country;

(5 rows affected)

Total execution time: 00:00:00.095

country,membership,website_purchase,pct_visits
Albania,Normal,646,0.349567099567
Albania,Premium,1185,0.641233766233
Albania,Not Logged In,17,0.009199134199
Antigua and Barbuda,Not Logged In,20,0.008818342151
Antigua and Barbuda,Normal,721,0.317901234567


The browsers the e-commerce site is accessed from will now be investigated. It is important to look at which browsers are being used by customers to ensure proper functionailty across all popular options. It is also important that the website meets all country-specific regulations and requirements that may vary by browser.

In [23]:
-- purchases per browser per country
SELECT TOP 10 country, Accessed_From, COUNT(1) AS purchases
FROM logs
GROUP BY country, Accessed_From
ORDER BY country;

(10 rows affected)

Total execution time: 00:00:00.061

country,Accessed_From,purchases
Albania,Mozilla Firefox,292
Albania,Chrome,299
Albania,Others,281
Albania,Android App,392
Albania,Microsoft Edge,171
Albania,Safari,180
Albania,IOS App,233
Antigua and Barbuda,Safari,210
Antigua and Barbuda,Chrome,401
Antigua and Barbuda,Microsoft Edge,189


These tables have been exported to Tableau to create insightful visualizations.